In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from order_of_magnitude import order_of_magnitude
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from itertools import combinations

from scipy.stats import mannwhitneyu, wilcoxon, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2, chi2_contingency, chisquare

from fitter import Fitter, get_common_distributions, get_distributions
from scipy.stats import kstest, ks_2samp

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, r2_score
import statsmodels.api as sm

In [21]:
data = pd.read_excel('Exercise.xlsx')
data.columns = list(data.iloc[0, :].values)
data.drop(index=0, inplace=True)
data.drop('Name', axis=1, inplace=True)
data.head()

C:\Users\Akshit Tayade\anaconda3\envs\akshit_venv\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Gender,Age,Height(cm),Weight(kg),Lifestyle,Exercise,Daily Calories Intake,Target,How motivated are you for being healthy\n(in %),Body Mass Index (BMI),Basal Metabolic Rate (BMR),Type,Body Fat Percentage (BFP),Body Type,Daily Calorie Need (Based on BMR and Exercise)\n(in Calorie),Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)\n(in Calorie),Calorie Difference (Daily Calorie Intake - TDEE),Suggestion
1,Male,18,152,50,1. Sedentary or light activity (e.g. - Office ...,1. Sedentary (little or no exercise),2067.152431,Loose Weight,80,21.641274,1392.866,Boy,17.878324,Normal,1671.4392,2131.08498,-0.03,LOW CARB DIET (25:35:40)
2,Male,18,152,54,1. Sedentary or light activity (e.g. - Office ...,2. Lightly active (light exercise/sports 1-3 d...,1705.730935,Maintain Weight,78,23.372576,1447.866,Boy,20.49259,Normal,1990.81575,2215.23498,-0.23,BALANCE DIET (55:25:20)
3,Male,18,152,58,1. Sedentary or light activity (e.g. - Office ...,3. Moderately active (moderate exercise/sports...,1910.142686,Gain Muscle,65,25.103878,1502.866,Boy,23.106856,Overweight,2329.4423,2299.38498,-0.18,BALANCE DIET (55:25:20)
4,Male,18,152,62,3. Vigorously active (Agricultural worker (non...,4. Very active (hard exercise/sports 6-7 days ...,2552.952908,Loose Weight,82,26.83518,1557.866,Boy,25.721122,Overweight,2687.31885,3505.1985,-0.271667,KETOGENIC DIET (5:35:60)
5,Male,18,152,66,1. Sedentary or light activity (e.g. - Office ...,1. Sedentary (little or no exercise),1826.086885,Maintain Weight,75,28.566482,1612.866,Boy,28.335388,Overweight,1935.4392,2467.68498,-0.26,BALANCE DIET (55:25:20)


In [22]:
# Mapping String to numeric values.
data['Gender']=data['Gender'].map({"Male":0,"Female":1})
data['Body Type']=data['Body Type'].map({"Underweight":0,"Normal":1,"Overweight":2,"Obese":3})
data['Type']=data['Type'].map({"Boy":0,"Girl":1,"Adult Male":2,"Adult Female":3})
data['Suggestion']=data['Suggestion'].map({"LOW CARB DIET (25:35:40)":0,"BALANCE DIET (55:25:20)":1,"ZONE DIET (40:30:30)":2,"KETOGENIC DIET (5:35:60)":3,"DEPLETION DIET (DYNAMIC)":4,"HIGH CARB DIET(60:20:20)":5})
data['Target']=data['Target'].map({"Loose Weight":0,"Maintain Weight":1,"Gain Muscle":2})
data['Lifestyle']=data['Lifestyle'].map({"1. Sedentary or light activity (e.g. - Office worker getting little or no exercise)":0,"2. Active or moderately active (Construction worker or person running one hour daily)":1,"3. Vigorously active (Agricultural worker (non mechanized) or person swimming two hours daily)":2})
data['Exercise']=data['Exercise'].map({"1. Sedentary (little or no exercise)":0,"2. Lightly active (light exercise/sports 1-3 days/ week)":1,"3. Moderately active (moderate exercise/sports 3-5days/week)":2,"4. Very active (hard exercise/sports 6-7 days a week)":3})

data = data.apply(pd.to_numeric)

In [23]:
data.drop('Body Type', axis=1, inplace=True)

In [24]:
sc_scaler = StandardScaler()
X_ = sc_scaler.fit_transform(data.iloc[:, :-1])
X_ = pd.DataFrame(X_, columns=list(data.columns)[:-1])
X_

,Gender,Age,Height(cm),Weight(kg),Lifestyle,Exercise,Daily Calories Intake,Target,How motivated are you for being healthy\n(in %),Body Mass Index (BMI),Basal Metabolic Rate (BMR),Type,Body Fat Percentage (BFP),Daily Calorie Need (Based on BMR and Exercise)\n(in Calorie),Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)\n(in Calorie),Calorie Difference (Daily Calorie Intake - TDEE)
0,-1.0,-1.704309,-1.46385,-1.243933,-1.224988,-1.360019,-0.948266,-1.224745,0.237504,-0.592030,-0.541999,-3.618136,-1.298428,-1.199687,-1.055461,-0.168609
1,-1.0,-1.704309,-1.46385,-1.006993,-1.224988,-0.473050,-1.385131,0.000000,0.043670,-0.342490,-0.321641,-3.618136,-1.017369,-0.530306,-0.926927,-1.307818
2,-1.0,-1.704309,-1.46385,-0.770054,-1.224988,0.413919,-1.138050,1.224745,-1.216250,-0.092950,-0.101283,-3.618136,-0.736309,0.179420,-0.798394,-1.023016
3,-1.0,-1.704309,-1.46385,-0.533114,1.224502,1.300887,-0.361060,-1.224745,0.431338,0.156589,0.119074,-3.618136,-0.455250,0.929492,1.043408,-1.545153
4,-1.0,-1.704309,-1.46385,-0.296174,-1.224988,-1.360019,-1.239652,0.000000,-0.247081,0.406129,0.339432,-3.618136,-0.174191,-0.646370,-0.541326,-1.478699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,1.0,1.565182,1.46385,0.296174,1.224502,1.300887,1.246541,0.000000,1.109757,-0.404250,-0.455130,0.904534,0.766252,0.411337,0.550863,1.255402
10076,1.0,1.565182,1.46385,0.533114,1.224502,-1.360019,1.136394,1.224745,1.303591,-0.230196,-0.301873,0.904534,0.922045,-1.048948,0.682325,0.913640
10077,1.0,1.565182,1.46385,0.770054,1.224502,-0.473050,-0.283890,-1.224745,-0.731665,-0.056142,-0.148616,0.904534,1.077837,-0.405851,0.813787,-1.250857
10078,1.0,1.565182,1.46385,1.006993,1.224502,0.413919,0.337951,0.000000,0.915923,0.117912,0.004641,0.904534,1.233630,0.265307,0.945249,-0.510371


In [25]:
x_train, x_test, y_train, y_test = train_test_split(X_, data.iloc[:, -1], test_size=0.2, random_state=1)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8064, 16), (2016, 16), (8064,), (2016,))

In [26]:
acc = []
r2score = []

for i in range(10, 16):

    features_selector_backward = sfs(  LogisticRegression(multi_class='multinomial', max_iter=1000),
                k_features=i, 
                forward=False, 
                verbose=0, 
                scoring='neg_mean_squared_error', cv=5, n_jobs=-1 )

    features_selector_backward.fit(x_train, y_train)

    X = X_[list(features_selector_backward.k_feature_names_)]

    X_train, X_test, Y_train, Y_test = train_test_split(X, data.iloc[:, -1], test_size=0.2, random_state=0)

    clf = LogisticRegression(C=10, multi_class='multinomial', n_jobs=-1).fit(X_train, Y_train)

    y_pred = clf.predict(X_test)
    
    acc.append(accuracy_score(Y_test, y_pred))
    r2score.append(r2_score(Y_test, y_pred))

    print("For {} features, score = {}".format(i, accuracy_score(Y_test, y_pred)))
    print("For {} features, R2score = {}".format(i, r2_score(Y_test, y_pred)))
    print()
    
print('- - - - - - - - - - - -')
print("Mean Acc = ", (sum(acc) / len(acc)))
print("Mean R2 = ", (sum(r2score) / len(r2score)))

For 10 features, score = 0.8120039682539683
For 10 features, R2score = 0.44449003394429987

For 11 features, score = 0.8095238095238095
For 11 features, R2score = 0.43395126983278787

For 12 features, score = 0.8100198412698413
For 12 features, R2score = 0.4318435170104854

For 13 features, score = 0.814484126984127
For 13 features, R2score = 0.4505790976531735

For 14 features, score = 0.8129960317460317
For 14 features, R2score = 0.4458952024925015

For 15 features, score = 0.8100198412698413
For 15 features, R2score = 0.4468319815246359

- - - - - - - - - - - -
Mean Acc =  0.8115079365079364
Mean R2 =  0.4422651837429807


In [27]:
features_selector_backward = sfs(  LogisticRegression(multi_class='multinomial', max_iter=1000),
                k_features=13, 
                forward=False, 
                verbose=0, 
                scoring='neg_mean_squared_error', cv=5, n_jobs=-1 )

features_selector_backward.fit(x_train, y_train)

SequentialFeatureSelector(estimator=LogisticRegression(max_iter=1000,
                                                       multi_class='multinomial'),
                          forward=False, k_features=(13, 13), n_jobs=-1,
                          scoring='neg_mean_squared_error')

In [28]:
features_selector_backward.k_feature_names_

('Age',
 'Height(cm)',
 'Weight(kg)',
 'Exercise',
 'Daily Calories Intake',
 'Target',
 'How motivated are you for being healthy\n(in %)',
 'Body Mass Index (BMI)',
 'Basal Metabolic Rate (BMR)',
 'Type',
 'Daily Calorie Need (Based on BMR and Exercise)\n(in Calorie)',
 'Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)\n(in Calorie)',
 'Calorie Difference (Daily Calorie Intake - TDEE)')

In [29]:
x_train_b = x_train[list(features_selector_backward.k_feature_names_)]
x_train_b.shape

(8064, 13)

In [30]:
log_reg_backward = sm.MNLogit(y_train.values.ravel(), x_train_b).fit()

Optimization terminated successfully.
         Current function value: 0.989104
         Iterations 8


In [31]:
log_reg_backward.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                      y   No. Observations:                 8064
Model:                        MNLogit   Df Residuals:                     7999
Method:                           MLE   Df Model:                           60
Date:                Sun, 11 Dec 2022   Pseudo R-squ.:                  0.3428
Time:                        00:45:54   Log-Likelihood:                -7976.1
converged:                       True   LL-Null:                       -12136.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================================================================
                                                                            y=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------
Age                                                                                -0.2873      0.076     -3.770      0.000      -0.437      -0.138
Height(cm)                                                                          2.2904      0.219     10.478      0.000       1.862       2.719
Weight(kg)                                                                         -4.8507      0.498     -9.746      0.000      -5.826      -3.875
Exercise                                                                            1.1609      0.332      3.494      0.000       0.510       1.812
Daily Calories Intake                                                              -4.6877      0.339    -13.832      0.000      -5.352      -4.023
Target                                                                              1.8214      0.061     29.874      0.000       1.702       1.941
How motivated are you for being healthy
(in %)                                      0.2041      0.047      4.311      0.000       0.111       0.297
Body Mass Index (BMI)                                                               3.6411      0.508      7.165      0.000       2.645       4.637
Basal Metabolic Rate (BMR)                                                          1.1369      0.473      2.402      0.016       0.209       2.065
Type                                                                                0.1780      0.065      2.718      0.007       0.050       0.306
Daily Calorie Need (Based on BMR and Exercise)
(in Calorie)                        -1.8728      0.556     -3.367      0.001      -2.963      -0.783
Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)
(in Calorie)     3.3333      0.248     13.418      0.000       2.846       3.820
Calorie Difference (Daily Calorie Intake - TDEE)                                    1.8110      0.195      9.273      0.000       1.428       2.194
---------------------------------------------------------------------------------------------------------------------------------------------------
                                                                            y=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------
Age                                                                                 0.0704      0.081      0.875      0.382      -0.087       0.228
Height(cm)                                                                          0.8953      0.238      3.757      0.000       0.428       1.362
Weight(kg)                                                                         -1.9461      0.490     -3.975      0.000      -2.9

In [32]:
log_reg_backward.aic

16082.267304826346

In [33]:
log_reg_backward.bic

16536.95302919657

## Resampling Methods
    - Using Cross validation

In [34]:
from sklearn.model_selection import cross_validate

lg_resultant = cross_validate(LogisticRegression(C=10, multi_class='multinomial', n_jobs=-1), 
                              x_train_b, 
                              y_train, 
                              cv=5,
                              return_train_score=True)

print("Mean Training Score = ", lg_resultant['train_score'].mean()*100)
print("Mean Testing Score = ", lg_resultant['test_score'].mean()*100)

Mean Training Score =  83.12563169942558
Mean Testing Score =  82.78777119526674


    - Using Bootstrap

In [35]:
train_score = []
test_score = []
n_times = 10

## Performing bootstrapping
for i in range(n_times):
    
    x_train, x_test, y_train, y_test = train_test_split(X_[list(features_selector_backward.k_feature_names_)], data.iloc[:, -1], test_size=0.2, random_state=42+i)
    
    #choose from different tunable hyper parameters
    clf = LogisticRegression(C=10, multi_class='multinomial', n_jobs=-1).fit(x_train, y_train)
    y_pred = clf.predict(x_test)
 
    #Measuring accuracy on Testing Data
    Accuracy = accuracy_score(y_test, y_pred)
    
    # Storing accuracy values
    test_score.append(Accuracy)
    train_score.append(accuracy_score(y_train, clf.predict(x_train)))
    
################################################
# Result of all bootstrapping trials
print("Mean Training Score = ", np.mean(train_score)*100)
print("Mean Testing Score = ", np.mean(test_score)*100)

Mean Training Score =  83.01215277777777
Mean Testing Score =  82.73809523809523
